In [3]:
import io
import os
import json
import distutils.dir_util
from collections import Counter

import numpy as np

def write_json(data, fname):
    def _conv(o):
        if isinstance(o, np.int64) or isinstance(o, np.int32):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname)
    distutils.dir_util.mkpath("./" + parent)
    with io.open("./" + fname, "w", encoding="utf8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)


def load_json(fname):
    with open(fname, encoding='utf8') as f:
        json_obj = json.load(f)

    return json_obj


def debug_json(r):
    print(json.dumps(r, ensure_ascii=False, indent=4))

In [32]:
# 채점

import numpy as np

class CustomEvaluator:
    def _idcg(self, l):
        return sum((1.0 / np.log(i + 2) for i in range(l)))
    
    def __init__(self):
        self._idcgs = [self._idcg(i) for i in range(101)]

    def _ndcg(self, gt, rec):
        dcg = 0.0
        for i, r in enumerate(rec):
            if r in gt:
                dcg += 1.0 / np.log(i + 2)

        return dcg / self._idcgs[len(gt)]

    def _eval(self, gt_fname, rec_fname):
        gt_playlists = load_json(gt_fname)
        gt_dict = {g["id"]: g for g in gt_playlists}
        rec_playlists = load_json(rec_fname)
        
        music_ndcg = 0.0
        tag_ndcg = 0.0
        
        for rec in rec_playlists:
            gt = gt_dict[rec["id"]]
            music_ndcg += self._ndcg(gt["songs"], rec["songs"][:100])
            tag_ndcg += self._ndcg(gt["tags"], rec["tags"][:10])
            
        music_ndcg = music_ndcg / len(rec_playlists)
        tag_ndcg = tag_ndcg / len(rec_playlists)
        score = music_ndcg * 0.85 + tag_ndcg * 0.15

        return music_ndcg, tag_ndcg, score

    def evaluate(self, gt_fname, rec_fname):
        try:
            music_ndcg, tag_ndcg, score = self._eval(gt_fname, rec_fname)
            print(f"Music nDCG: {music_ndcg:.6}")
            print(f"Tag nDCG: {tag_ndcg:.6}")
            print(f"Score: {score:.6}")
        except Exception as e:
            print(e)

In [5]:
import pickle
import numpy as np
import pandas as pd
import scipy.sparse as spr
from collections import Counter
from implicit.evaluation import  *
from implicit.als import AlternatingLeastSquares as ALS
from scipy.sparse import hstack, vstack
from tqdm.auto import tqdm

song_meta = pd.read_json("./../data/song_meta.json")
train = pd.read_json("./../data/train.json")
test = pd.read_json("./../data/val.json")

In [6]:
train.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000


In [7]:
genre_onehot = np.load("./../data/genre_onehot.npy")
year_onehot = np.load("./../data/year_onehot.npy")

genre_onehot.shape

(138086, 254)

In [8]:
# genre_header = ['발라드', '발라드', "'80", "'90", "'00", "'10-", '댄스', '댄스', "'80", "'90", "'00", "'10-", '랩/힙합', '랩/힙합', '랩 스타일',
#  '보컬 스타일','언더그라운드 힙합','시대별','R&B/Soul','R&B/Soul''어반','R&B','인디음악','인디음악','포크','록','일렉','힙합','발라드',"'90","'00","'10-",
#  '록/메탈','록/메탈',"'70","'80","'90","'00","'10-",'성인가요','성인가요','신세대트로트','전설의트로트','뽕짝트로트','트로트메들리',"트로트'60-'70","트로트'80-'90",
#  "트로트'00-","성인가요'80-'90","성인가요'00-",'포크/블루스','포크/블루스',"'60-'70","'80-'90","'00","'10-",'POP','POP','얼터너티브팝','올디스','월드팝',
#  "'60-'70","'80-'90","'00","'10-",'록/메탈','록/메탈','모던록','얼터너티브록','프로그레시브/아트록','하드록','헤비메탈','뉴메탈/하드코어','포스트록',"'60","'70",
#  "'80","'90","'00","'10-",'일렉트로니카','일렉트로니카','일렉트로니카','하우스','클럽뮤직','일렉트로닉팝',"'80","'90","'00","'10-",'랩/힙합','랩/힙합','팝랩',
#  '얼터너티브힙합','갱스터/하드코어랩','East&West',"'80","'90","'00","'10-",'R&B/Soul','R&B/Soul','컨템포러리 R&B','소울','어반',"'60-'70","'80-'90",
#  "'00","'10-",'포크/블루스/컨트리','포크/블루스/컨트리','포크','블루스','컨트리',"'60-'70","'80-'90","'00-",'OST','OST','국내영화','국외영화','국내드라마',
#  '국외드라마','애니메이션/게임','국내뮤지컬','국외뮤지컬','애니메이션/웹툰','게임','클래식','클래식','관현악','교향곡','실내악','협주곡','독주곡','오페라','크로스오버',
#  '현대음악','성악/합창곡','발레/무용','지휘/연주자','컴필레이션','교향/관현악','오페라/성악','재즈','재즈','보컬재즈','애시드/퓨전/팝','Bop','보사노바', 'J-Jazz', '라틴재즈',
#  '빅밴드/스윙', '악기별', '컴필레이션', '뉴에이지', '뉴에이지', '이지리스닝', 'J-Newage', '기능성음악', '기타', '피아노', '컴필레이션', 'J-POP', 'J-POP', 'POP', '록', '일렉트로니카',
#  '랩/힙합', 'R&B/Soul', '시부야케이', '뉴에이지', '재즈', '힙합/R&B', "'80-'90", "'00-'10", '월드뮤직', '월드뮤직', '샹송/프렌치팝', '칸초네/이탈리안팝', '중국음악',
#  '켈틱/아이리시', '브라질', '탱고/플라멩코', '라틴', '레게', '파두', 'CCM', 'CCM', '국내CCM', '국외CCM', '워십', '찬송가', '성가', '연주곡', '어린이', '성경', 'NEW',
#  '어린이/태교', '어린이/태교', '창작동요', '영어동요', '영어동요', '영어동요', '영어동화', '만화', '자장가', '태교동화', '태교', '릴렉싱&힐링', '명작동화', '교과서동요',
#  '전래동요', '창작동화', '어린이클래식', '종교음악', '종교음악', '불교', '국악', '국악', '크로스오버', '국악가요', '민요', '판소리/단가', '풍물/사물놀이', '아이돌', '아이돌',
#  '남자 아이돌', '여자 아이돌', '랩/힙합', '발라드', '댄스', '일렉트로니카', '일렉트로니카', '댄스 스타일', '팝 스타일', '시대별', 'EDM', 'EDM', '하우스', '클럽',
#  'DJ', '뮤직테라피', '뮤직테라피', 'ASMR/자연', '힐링/명상/요가', '집중력', '숙면유도', '반려동물', '뮤지컬', '뮤지컬', '국내뮤지컬', '국외뮤지컬', '크리스마스']

# year_header = [0, 1900, 1919, 1927, 1931, 1936, 1939, 1941, 1945, 1946, 1947, 1949, 1950, 1951, 1952, 1953, 1954, 1955,
#  1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975,
#  1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995,
#  1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
#  2016, 2017, 2018, 2019, 2020, 2022]

genre_header = [ 'genre_' + str(i) for i in range(254) ]
year_header  = [ 'year_' + str(i) for i in range(84) ]

genre_onehot = pd.DataFrame(genre_onehot, columns=genre_header).astype('int')
year_onehot = pd.DataFrame(year_onehot, columns=year_header).astype('int')

genre_onehot.head()

,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,...,genre_244,genre_245,genre_246,genre_247,genre_248,genre_249,genre_250,genre_251,genre_252,genre_253
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
train['istrain'] = 1
test['istrain'] = 0

n_train = len(train)
n_test = len(test)

# train + test
plylst = pd.concat([train, test], ignore_index=True)

# playlist id
plylst["nid"] = range(n_train + n_test)

# id <-> nid
plylst_id_nid = dict(zip(plylst["id"],plylst["nid"]))
plylst_nid_id = dict(zip(plylst["nid"],plylst["id"]))

In [10]:
plylst.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,istrain,nid
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,1,0
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,1,1
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,1,2
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,1,3
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,1,4


In [11]:
plylst_tag = plylst['tags']
tag_counter = Counter([tg for tgs in plylst_tag for tg in tgs])
tag_dict = {x: tag_counter[x] for x in tag_counter}

tag_id_tid = dict()
tag_tid_id = dict()
for i, t in enumerate(tag_dict):
    tag_id_tid[t] = i
    tag_tid_id[i] = t

#####
    
n_tags = len(tag_dict)

plylst_song = plylst['songs']
song_counter = Counter([sg for sgs in plylst_song for sg in sgs])
song_dict = {x: song_counter[x] for x in song_counter}

song_id_sid = dict()
song_sid_id = dict()
for i, t in enumerate(song_dict):
    song_id_sid[t] = i
    song_sid_id[i] = t

n_songs = len(song_dict)

In [12]:
plylst['songs_id'] = plylst['songs'].map(lambda x: [song_id_sid.get(s) for s in x if song_id_sid.get(s) != None])
plylst['tags_id'] = plylst['tags'].map(lambda x: [tag_id_tid.get(t) for t in x if tag_id_tid.get(t) != None])

plylst_use = plylst[['istrain','nid','updt_date','songs_id','tags_id']]

# 곡의 개수와 태그의 개수를 할당
plylst_use.loc[:,'num_songs'] = plylst_use['songs_id'].map(len)
plylst_use.loc[:,'num_tags'] = plylst_use['tags_id'].map(len)
plylst_use = plylst_use.set_index('nid')

C:\Users\green198800\Anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\green198800\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [13]:
# train test split
plylst_train = plylst_use.iloc[:n_train,:]
plylst_test = plylst_use.iloc[n_train:,:]

In [14]:
# training set
row = np.repeat(range(n_train), plylst_train['num_songs'])
col = [song for songs in plylst_train['songs_id'] for song in songs]
dat = np.repeat(1, plylst_train['num_songs'].sum())
train_songs_A = spr.csr_matrix((dat, (row, col)), shape=(n_train, n_songs))

row = np.repeat(range(n_train), plylst_train['num_tags'])
col = [tag for tags in plylst_train['tags_id'] for tag in tags]
dat = np.repeat(1, plylst_train['num_tags'].sum())
train_tags_A = spr.csr_matrix((dat, (row, col)), shape=(n_train, n_tags))

train_genre_A = spr.csr_matrix((genre_onehot.loc[:n_train - 1]))
train_year_A  = spr.csr_matrix((year_onehot.loc[:n_train - 1]))

# test set
row = np.repeat(range(n_test), plylst_test['num_songs'])
col = [song for songs in plylst_test['songs_id'] for song in songs]
dat = np.repeat(1, plylst_test['num_songs'].sum())
test_songs_A = spr.csr_matrix((dat, (row, col)), shape=(n_test, n_songs))

row = np.repeat(range(n_test), plylst_test['num_tags'])
col = [tag for tags in plylst_test['tags_id'] for tag in tags]
dat = np.repeat(1, plylst_test['num_tags'].sum())
test_tags_A = spr.csr_matrix((dat, (row, col)), shape=(n_test, n_tags))

test_genre_A = spr.csr_matrix((genre_onehot.loc[n_train:]))
test_year_A  = spr.csr_matrix((year_onehot.loc[n_train:]))

In [15]:
train_songs_A.shape

(115071, 638336)

In [16]:
train_genre_A.shape

(115071, 254)

In [17]:
train_year_A.shape

(115071, 84)

In [18]:
train_merged_feature_csr = hstack([train_songs_A, train_tags_A, train_genre_A, train_year_A])
test_merged_feature_csr  = hstack([test_songs_A, test_tags_A, test_genre_A, test_year_A])

train_songs_A_T = train_songs_A.T.tocsr()
train_tags_A_T  = train_tags_A.T.tocsr()

whole_datasets_csr       = vstack([train_merged_feature_csr, test_merged_feature_csr])
whole_datasets_csr_T     = whole_datasets_csr.T.tocsr()

In [19]:
whole_datasets_csr_T.shape

(668871, 138086)

In [ ]:
# def rec(pids):
#     res = []

#     for pid in tqdm(pids):
#         p = np.zeros((n_songs,1))
#         p[plylst_test.loc[pid,'songs_id']] = 1

#         val = train_songs_A.dot(p).reshape(-1)

#         songs_already = plylst_test.loc[pid, "songs_id"]
#         tags_already = plylst_test.loc[pid, "tags_id"]

#         cand_song = train_songs_A_T.dot(val)
#         cand_song_idx = cand_song.reshape(-1).argsort()[-250:][::-1]

#         cand_song_idx = cand_song_idx[np.isin(cand_song_idx, songs_already) == False][:100]
#         rec_song_idx = [song_sid_id[i] for i in cand_song_idx]

#         cand_tag = train_tags_A_T.dot(val)
#         cand_tag_idx = cand_tag.reshape(-1).argsort()[-15:][::-1]

#         cand_tag_idx = cand_tag_idx[np.isin(cand_tag_idx, tags_already) == False][:10]
#         rec_tag_idx = [tag_tid_id[i] for i in cand_tag_idx]

#         res.append({
#             "id": plylst_nid_id[pid],
#             "songs": rec_song_idx,
#             "tags": rec_tag_idx
#         })
        
#     return res

# res = rec(plylst_test.index)

In [41]:
n_factor = 128

als_model = ALS(factors=n_factor, regularization=0.08)
als_model.fit(whole_datasets_csr_T * 15)

song_model = ALS(use_gpu=False)
tag_model = ALS(use_gpu=False)
song_model.user_factors = als_model.user_factors
tag_model.user_factors = als_model.user_factors

song_model.item_factors = als_model.item_factors[:n_songs]
tag_model.item_factors  = als_model.item_factors[n_songs:n_songs + n_tags]

print(song_model.item_factors.shape)
print(tag_model.item_factors.shape)


(638336, 128)
(30197, 128)


In [42]:
res = []

for u in tqdm(range(test_merged_feature_csr.shape[0])):
    songs_already = plylst_test.iloc[u]["songs_id"]
    song_rec = song_model.recommend(u, test_songs_A, N=200)

    song_rec = np.array([song_sid_id[x[0]] for x in song_rec])    
    song_rec = song_rec[np.isin(song_rec, songs_already) == False][:100]
    
    tags_already = plylst_test.iloc[u]["tags_id"]
    tag_rec = tag_model.recommend(u, test_tags_A, N=20)
    tag_rec = np.array([tag_tid_id[x[0]] for x in tag_rec])
    tag_rec = tag_rec[np.isin(tag_rec, tags_already) == False][:10]
    
    res.append({
            # train test의 vconcat 된 matrix에서 id를 추출하는 것이기 때문에 n_train 이후부터 test 데이터 셋임
            # 따라서 u + n_train이 각각의 test 셋에서의 id
            "id": plylst_nid_id[u + n_train],
            "songs": song_rec.tolist(),
            "tags": tag_rec.tolist()
        })

In [43]:
# os.mkdir("./year_genre_onehot")
write_json(res, "./year_genre_onehot/results_wls_128.json")

In [40]:
evaluator = CustomEvaluator()
evaluator.evaluate("./../best_performance_result/results.json", "./year_genre_onehot/results_wls_256.json")

0.02958963787636777 0.1388624438735545
0.02958963787636777 0.1388624438735545
0.02958963787636777 0.4225749983705863
0.07219304284266125 0.4225749983705863
0.07219304284266125 0.4225749983705863
0.6844726105726783 0.99626491802783
0.8140454559831216 1.311145048695461
0.8140454559831216 1.4441970588311899
0.8140454559831216 1.4441970588311899
0.8140454559831216 1.4441970588311899
0.8140454559831216 1.4441970588311899
0.8140454559831216 1.66428882512927
0.8140454559831216 1.7337200470660472
0.8140454559831216 1.7337200470660472
0.8408883259505191 1.7973408352649989
0.8408883259505191 1.7973408352649989
0.9005264843892018 1.9768179403508161
0.9005264843892018 1.9768179403508161
0.9005264843892018 2.0552162093294917
0.9005264843892018 2.0552162093294917
0.9005264843892018 2.0552162093294917
0.9005264843892018 2.3524878057715486
1.0013373970402613 2.3524878057715486
1.0013373970402613 2.3524878057715486
1.1405550191034846 2.926177725428792
1.1405550191034846 2.926177725428792
1.140555019103

17.64085103954702 107.85154746797767
17.64085103954702 107.85154746797767
17.64085103954702 107.93669058561929
17.70490523217251 108.24997540252103
17.719481966040128 108.24997540252103
17.719481966040128 108.61883107465968
17.817941239029473 109.04092603503051
17.817941239029473 109.04092603503051
17.817941239029473 109.10454682322946
17.817941239029473 109.18968994087108
17.817941239029473 109.18968994087108
17.817941239029473 109.60990716766665
17.817941239029473 109.6735279558656
17.817941239029473 109.76831632023514
17.913255680954506 109.97315056775373
17.95103495646599 110.20680137599685
17.95103495646599 110.20680137599685
17.98073599961109 110.6508987038101
17.98073599961109 110.6508987038101
17.98073599961109 110.6508987038101
17.98073599961109 110.8144400904304
17.98073599961109 110.95330253430394
17.988501932475117 111.28060913030521
18.053797689575454 111.76532111933072
18.088468951362298 112.20941844714397
18.088468951362298 112.77646140531076
18.11443881920516 113.073733

32.963057177575685 209.3709875570229
32.963057177575685 209.59107932332097
32.963057177575685 209.59107932332097
33.00079919472749 209.59107932332097
33.00079919472749 209.65470011151993
33.38455088921323 209.65470011151993
33.39266943777381 209.88449572159462
33.39266943777381 209.88449572159462
33.45310031663523 210.44828417928485
33.45310031663523 210.44828417928485
33.63535569769606 210.5430725436544
33.66058616043169 210.87321019310153
33.66058616043169 210.87321019310153
33.66058616043169 210.87321019310153
33.66058616043169 211.07804444062012
33.66058616043169 211.3020500021353
33.66058616043169 211.36567079033426
33.66058616043169 211.36567079033426
33.66058616043169 211.36567079033426
33.66058616043169 211.56085979112493
33.66058616043169 211.7848653526401
33.78850790085028 212.0049571189382
33.78850790085028 212.0049571189382
33.78850790085028 212.0049571189382
33.78850790085028 212.0049571189382
33.811822525106756 212.3643657060352
33.99545605706588 212.43061992948958
34.071

47.87109366018423 302.48540818518853
47.87109366018423 302.48540818518853
47.87109366018423 302.5516624086429
47.87109366018423 302.6179166320973
47.87843232495491 302.6179166320973
47.87843232495491 302.696314901076
47.87843232495491 302.98657468604557
47.87843232495491 302.98657468604557
47.87843232495491 302.98657468604557
47.87843232495491 302.98657468604557
47.885652373057745 303.50657148404116
47.924482577806735 304.0422801777416
47.924482577806735 304.0422801777416
47.924482577806735 304.26237194403967
47.924482577806735 304.26237194403967
47.924482577806735 304.70646927185294
47.924482577806735 304.70646927185294
47.924482577806735 305.02134940252057
47.93235561128207 305.09471332461993
47.93235561128207 305.09471332461993
48.180235536573704 305.314805090918
48.180235536573704 305.62968522158565
48.180235536573704 305.62968522158565
48.180235536573704 305.62968522158565
48.180235536573704 305.62968522158565
48.366099275416516 305.7397311047347
48.366099275416516 305.73973110473

61.721266160502935 405.1225171162757
61.721266160502935 405.39836427044753
61.728470256767146 405.53141628058324
61.86262539532102 406.00041637390393
61.86262539532102 406.00041637390393
61.86262539532102 406.220508140202
61.92839861616845 406.28412892840095
61.9450534939045 406.50422069469903
61.9450534939045 407.1720469836101
61.9450534939045 407.1720469836101
61.9450534939045 407.1720469836101
61.9450534939045 407.1720469836101
61.9450534939045 407.1720469836101
61.9450534939045 407.1720469836101
61.9450534939045 407.1720469836101
62.078143011219176 407.5653253832564
62.078143011219176 408.19786686319736
62.078143011219176 408.2614876513963
62.078143011219176 408.7789489013089
62.13014996371197 409.0215667565939
62.1497938661953 409.13161263974297
62.1497938661953 409.13161263974297
62.23069536002714 409.4913225515948
62.237868235713734 409.8096621005542
62.24553558415232 409.8096621005542
62.24553558415232 409.8096621005542
62.24553558415232 410.0585704275768
62.24553558415232 410.

...

425.9130621625864 2979.613577060174
425.9130621625864 2980.017059726775
425.9130621625864 2980.017059726775
425.9130621625864 2980.086490948712
425.9130621625864 2980.086490948712
425.9130621625864 2980.086490948712
425.9130621625864 2980.181279313081
425.9130621625864 2980.181279313081
425.9284898735156 2980.25967758206
425.9817824962137 2980.3697234652086
425.9817824962137 2980.3697234652086
425.9817824962137 2980.3697234652086
426.17756796111325 2980.5898152315067
426.17756796111325 2980.5898152315067
426.18514413248863 2980.5898152315067
426.18514413248863 2980.5898152315067
426.2011152004288 2980.5898152315067
426.2184582343773 2980.6592464534433
426.2184582343773 2980.7326103755427
426.55073593450584 2981.1921429986432
426.55073593450584 2981.1921429986432
426.55073593450584 2981.551097208815
426.6478547273589 2981.771188975113
426.6589050091341 2981.771188975113
426.6589050091341 2981.881234858262
426.6589050091341 2982.164947412759
426.6589050091341 2982.164947412759
426.658905

439.251051249969 3083.4423508157133
439.251051249969 3083.911350909034
439.251051249969 3083.911350909034
439.251051249969 3083.911350909034
439.251051249969 3083.911350909034
439.2725178706463 3084.365093483575
439.2876302525367 3084.4434917525537
439.2876302525367 3084.667497314069
439.3194610894261 3084.8095163712464
439.3194610894261 3085.062811775357
439.3194610894261 3085.062811775357
439.3194610894261 3085.062811775357
439.3194610894261 3085.484906735728
439.3194610894261 3085.910115169354
439.5128715968543 3086.225277719831
439.5128715968543 3086.225277719831
439.5288205105082 3086.445369486129
439.5288205105082 3086.445369486129
439.5288205105082 3086.5148007080656
439.72392069660646 3086.6733077478066
439.72392069660646 3086.9717977830833
439.72392069660646 3087.160241935211
439.72392069660646 3087.160241935211
439.74013450571795 3087.3048944276443
439.74013450571795 3087.3048944276443
439.8273169674913 3087.8116781287313
439.8273169674913 3087.87529891693
439.8273169674913 3

453.63651903364445 3190.0568215341823
453.63651903364445 3190.0568215341823
453.63651903364445 3190.0568215341823
453.63651903364445 3190.120442322381
453.63651903364445 3190.120442322381
453.814349713567 3190.278949362122
453.814349713567 3190.278949362122
453.814349713567 3190.278949362122
453.82268756037195 3190.388995245271
453.82268756037195 3190.719132894718
453.82268756037195 3190.870895085796
453.82268756037195 3190.9492933547745
453.82268756037195 3190.9492933547745
453.82268756037195 3190.9492933547745
453.82268756037195 3190.9492933547745
453.82268756037195 3191.018724576711
453.82268756037195 3191.3050705664637
453.82268756037195 3191.5251623327617
453.84656682200233 3192.231067734802
453.93870870063455 3192.5612053842488
453.93870870063455 3192.5612053842488
453.93870870063455 3192.5612053842488
453.93870870063455 3192.998557863399
453.93870870063455 3192.998557863399
453.93870870063455 3192.998557863399
454.14881611028187 3193.257367652389
454.14881611028187 3193.36741353

468.62944533743877 3294.965045489198
468.62944533743877 3295.248758043695
468.62944533743877 3295.3150122671495
468.62944533743877 3295.629892397817
468.64872718793913 3295.6993236197536
468.76743813519874 3296.014203750421
468.76743813519874 3296.014203750421
468.76743813519874 3296.07782453862
468.7788912244616 3296.3641705283726
468.7788912244616 3296.449313646014
468.7788912244616 3296.449313646014
468.7788912244616 3296.449313646014
468.7788912244616 3296.6694054123122
468.7788912244616 3296.6694054123122
468.7788912244616 3296.733026200511
468.7788912244616 3296.9502869133635
468.8344336871277 3297.647225268362
468.85001582537154 3297.647225268362
468.85001582537154 3297.782910713753
468.90429044127035 3298.4823846168347
468.90429044127035 3298.4823846168347
468.90429044127035 3298.5607828858133
468.91925000120807 3298.5607828858133
468.91925000120807 3298.5607828858133
468.95270940873814 3298.645926003455
468.95270940873814 3298.645926003455
468.95270940873814 3298.645926003455


485.8711241677771 3411.2634496043165
485.8711241677771 3411.2634496043165
485.8711241677771 3411.659841477218
485.8711241677771 3411.659841477218
486.1904772812592 3412.41047877327
486.1904772812592 3412.41047877327
486.1904772812592 3412.41047877327
486.1904772812592 3412.4799099952065
486.3570628885981 3412.9336525697477
486.3570628885981 3412.9336525697477
486.37277419652287 3413.334323087848
486.59889688236814 3414.0337969909297
486.59889688236814 3414.0337969909297
486.59889688236814 3414.0337969909297
486.59889688236814 3414.5320869268457
486.59889688236814 3414.6421328099946
486.59889688236814 3414.6421328099946
486.7420149790354 3414.947367693934
486.8559569472943 3415.587676687028
486.8559569472943 3415.7901599191005
486.8559569472943 3415.7901599191005
486.8559569472943 3415.7901599191005
486.8559569472943 3415.88494828347
486.8559569472943 3415.9485690716688
486.8559569472943 3415.9485690716688
486.92799120095094 3416.3075232818405
486.92799120095094 3416.619572359062
486.92

In [44]:
evaluator = CustomEvaluator()
evaluator.evaluate("./../best_performance_result/results.json", "./year_genre_onehot/results_wls_128.json")

0.021890531479706928 0.1388624438735545
0.021890531479706928 0.1388624438735545
0.021890531479706928 0.43735247915031
0.0488334618902219 0.5036067026046991
0.0488334618902219 0.5036067026046991
0.7079725501258208 1.0136758340629912
0.8592615975542575 1.479953305826632
0.8592615975542575 1.479953305826632
0.8592615975542575 1.479953305826632
0.8592615975542575 1.479953305826632
0.8592615975542575 1.479953305826632
0.8592615975542575 1.7000450721247122
0.8592615975542575 1.7000450721247122
0.8592615975542575 1.7000450721247122
0.894034211753783 1.794833436494263
0.894034211753783 1.794833436494263
0.9481350384512687 2.154524443400605
0.9481350384512687 2.154524443400605
0.9481350384512687 2.5454518276008264
0.9559009713152962 2.7288616328492266
0.9559009713152962 2.7288616328492266
0.9559009713152962 3.0693813257589837
1.082871148541702 3.1330021139579354
1.082871148541702 3.2114003829366107
1.2351836703587569 3.7920023830484126
1.2351836703587569 3.7920023830484126
1.2351836703587569 4.

21.034831587100904 123.30269612069475
21.034831587100904 123.47084840716366
21.034831587100904 123.5344691953626
21.034831587100904 123.64451507851165
21.141252827449943 124.02329841718758
21.171960791224276 124.16216086106112
21.171960791224276 124.16216086106112
21.221398098900103 124.69741517783619
21.221398098900103 124.69741517783619
21.221398098900103 124.69741517783619
21.221398098900103 124.69741517783619
21.221398098900103 124.80746106098523
21.221398098900103 125.20073946063154
21.286818481121035 125.64483678844479
21.29423101287637 126.16801058492275
21.29423101287637 126.8259092072965
21.312690029959985 127.21050297971027
21.312690029959985 127.27675720316466
21.312690029959985 127.59191975364165
21.320430565420246 127.65817397709604
21.320430565420246 127.65817397709604
21.37891636087868 128.22354411950278
21.39483586863229 128.47245244652538
21.39483586863229 128.47245244652538
21.40236893696026 128.54188366846216
21.45006514324261 128.7501773342725
21.45006514324261 128.

35.99255355350373 216.00163132400797
35.99255355350373 216.00163132400797
35.99255355350373 216.00163132400797
35.99255355350373 216.00163132400797
36.017554160161396 216.80232509041787
36.02977833744307 217.19560349006417
36.06754246198501 217.33446593393774
36.06754246198501 217.33446593393774
36.06754246198501 217.44451181708678
36.06754246198501 217.44451181708678
36.06754246198501 217.44451181708678
36.11504745345653 218.00181164135316
36.11504745345653 218.06806586480755
36.11504745345653 218.2108610088437
36.14211020170441 218.29600412648531
36.14211020170441 218.44740146758133
36.14211020170441 218.44740146758133
36.325091542429675 219.04627657255534
36.325091542429675 219.21946320590357
36.325091542429675 219.21946320590357
36.49358477699372 219.7427145043757
36.526516213978496 219.81214572631248
36.526516213978496 219.96672006589088
36.55412847669055 220.2093379211759
36.6087614283904 220.43913353125058
36.6087614283904 220.83915677955983
36.6087614283904 221.13642837600187
3

54.558934946248904 332.0354618277798
54.558934946248904 332.0354618277798
54.558934946248904 332.10882574987914
54.558934946248904 332.5529230776924
54.61388862612117 332.9970204055057
54.61388862612117 333.0632746289601
54.61388862612117 333.28336639525816
54.647811978855536 333.42538545243576
54.70257662652246 333.85770358470677
54.70257662652246 333.85770358470677
54.73502425236422 334.2166577948784
54.73502425236422 334.2166577948784
54.74232771824422 334.2166577948784
54.76704858852247 334.2166577948784
54.79092785015288 334.2166577948784
54.79092785015288 334.75554348706123
54.79092785015288 335.0020940425088
54.80530459820301 335.0754579646082
54.87792043453241 335.60256446124873
54.87792043453241 336.0563070357899
54.87792043453241 336.0563070357899
54.87792043453241 336.0563070357899
54.87792043453241 336.0563070357899
54.87792043453241 336.0563070357899
54.87792043453241 336.38644468523705
54.87792043453241 336.38644468523705
54.87792043453241 336.38644468523705
54.8779204345

71.03670406228127 441.52545850504777
71.12124167344813 441.7591093132909
71.12124167344813 441.8691551964399
71.12124167344813 441.8691551964399
71.12124167344813 442.22810940661157
71.12124167344813 442.22810940661157
71.12124167344813 442.3709045506477
71.12124167344813 442.3709045506477
71.12124167344813 442.73564880937886
71.12124167344813 442.73564880937886
71.12124167344813 443.0505289400465
71.1486314357648 443.3635124321934
71.1486314357648 443.3635124321934
71.1486314357648 443.3635124321934
71.16506759075459 443.6472249866904
71.16506759075459 443.79598889253094
71.16506759075459 444.18926729217725
71.16506759075459 444.18926729217725
71.16506759075459 444.2528880803762
71.1850719501924 444.7066306549174
71.1850719501924 445.19545987674235
71.19445944347757 445.4857196617119
71.19445944347757 445.4857196617119
71.19445944347757 445.8040592106713
71.33765786674283 445.9429216545449
71.38055360982214 445.9429216545449
71.38055360982214 446.00917587799927
71.50408916452476 446.3

...

516.0978619484104 3439.2420176805444
516.0978619484104 3439.3965920201226
516.0978619484104 3439.3965920201226
516.0978619484104 3439.469955942222
516.1129832885729 3439.9931297386997
516.1129832885729 3440.166796410048
516.1129832885729 3440.3037811203462
516.1129832885729 3440.436833130482
516.1129832885729 3440.6320221312726
516.1129832885729 3440.6320221312726
516.1129832885729 3440.6320221312726
516.1129832885729 3441.2115224846675
516.1129832885729 3441.2115224846675
516.240157064729 3441.6403622937014
516.240157064729 3441.6403622937014
516.240157064729 3441.913032449391
516.240157064729 3441.9766532375897
516.240157064729 3441.9766532375897
516.240157064729 3441.9766532375897
516.2475137504938 3442.2751432728664
516.2475137504938 3442.7288858474076
516.2475137504938 3443.0125984019046
516.2475137504938 3443.0125984019046
516.2475137504938 3443.0125984019046
516.3334258732513 3443.481598495225
516.4715702607352 3444.1810723983067
516.4715702607352 3444.1810723983067
516.47157026

530.7575370508658 3561.121172379119
530.809257512667 3561.5660254086124
530.8265666345393 3561.7743190744227
530.8265666345393 3561.7743190744227
530.9322014387707 3562.0805577179067
530.9322014387707 3562.0805577179067
530.9322014387707 3562.0805577179067
530.9322014387707 3562.253744351255
530.9940985640599 3562.4025082570956
531.0015900507832 3562.957407169738
531.0015900507832 3563.0236613931925
531.0015900507832 3563.187880979499
531.0015900507832 3563.187880979499
531.0015900507832 3563.2541352029534
531.0015900507832 3563.2541352029534
531.0015900507832 3563.32356642489
531.0088935166632 3563.478140764468
531.0088935166632 3563.478140764468
531.0088935166632 3563.909682809881
531.0088935166632 3563.909682809881
531.0088935166632 3563.909682809881
531.0088935166632 3564.2031384982783
531.0088935166632 3564.452046825301
531.0088935166632 3564.6545300573734
531.0758603471028 3565.417624748654
531.0758603471028 3565.417624748654
531.0758603471028 3565.417624748654
531.0912682310017 

548.3338021741117 3683.9750967990085
548.3338021741117 3684.5072376425283
548.3526995256756 3684.570858430727
548.3526995256756 3684.6656467950966
548.3673596773203 3684.8490566003447
548.3673596773203 3684.8490566003447
548.3750035757423 3685.245794418283
548.3750035757423 3685.245794418283
548.3750035757423 3685.245794418283
548.7182230331147 3685.85758965564
548.7182230331147 3686.311332230181
548.8623127735008 3686.531423996479
548.8623127735008 3686.531423996479
548.8623127735008 3686.531423996479
548.8623127735008 3687.187727305087
548.9379637404068 3687.8631433306427
548.9465617707085 3688.3766718803927
548.9743127858063 3688.820769208206
548.9743127858063 3688.820769208206
548.9743127858063 3689.1797234183778
548.9743127858063 3689.1797234183778
548.9743127858063 3689.1797234183778
548.9743127858063 3689.5679674711446
549.1378033804597 3689.6315882593435
549.1378033804597 3689.6952090475424
549.1378033804597 3689.9124697603947
549.1378033804597 3689.985833682494
549.13780338045

566.5282001323308 3807.125468294162
566.5282001323308 3807.5944683874827
566.546682184362 3807.8968722179766
566.546682184362 3807.982015335618
566.546682184362 3807.982015335618
566.5666722945077 3807.982015335618
566.5666722945077 3807.982015335618
566.5666722945077 3807.982015335618
566.5666722945077 3807.982015335618
566.574226718181 3807.982015335618
566.574226718181 3807.982015335618
566.574226718181 3807.982015335618
566.574226718181 3808.240825124608
566.5967561938177 3808.240825124608
566.5967561938177 3808.240825124608
566.5967561938177 3808.240825124608
566.5967561938177 3808.240825124608
566.5967561938177 3808.511913595581
566.6059939532141 3808.725997309951
566.6553125911045 3808.9730279033984
566.6553125911045 3808.9730279033984
566.7907660200299 3809.4171252312117
566.9392409213024 3809.971268442174
566.9392409213024 3809.971268442174
566.9392409213024 3810.0406996641104
566.9392409213024 3810.1043204523094
566.9392409213024 3810.643206144492
566.971134068778 3811.112206

Music nDCG: 0.0253858
Tag nDCG: 0.170193
Score: 0.0471069


In [39]:
evaluator = CustomEvaluator()
evaluator.evaluate("./../best_performance_result/results.json", "./year_genre_onehot/results_wls_64.json")

0.01964088693171018 0.1388624438735545
0.01964088693171018 0.1388624438735545
0.01964088693171018 0.4252084336260237
0.04196777126265269 0.5988751049740155
0.04196777126265269 0.5988751049740155
0.6625184645405087 1.2289177861186302
0.8088446952865219 1.6007528385492584
0.8088446952865219 1.6007528385492584
0.8088446952865219 1.6007528385492584
0.8088446952865219 1.6007528385492584
0.8088446952865219 1.66437362674821
0.8088446952865219 1.7744195098972502
0.8088446952865219 1.8844653930462902
0.8088446952865219 1.9538966149830674
0.8314729976995413 2.1843704247437845
0.8314729976995413 2.1843704247437845
0.886735968133646 2.392664090554116
0.886735968133646 2.392664090554116
0.886735968133646 2.5444262816321515
0.89628767278581 2.6138575035689287
0.89628767278581 2.6138575035689287
0.89628767278581 2.9479089482331444
1.0118676094363799 3.1687375112659315
1.0118676094363799 3.232358299464883
1.2442837729523606 3.914039135683827
1.2442837729523606 3.914039135683827
1.2442837729523606 4.38

25.642384367065286 145.3660796449084
25.642384367065286 145.3660796449084
25.642384367065286 145.45122276255003
25.83636349050628 145.93986745173814
25.85337541973745 146.31836837060473
25.85337541973745 146.31836837060473
25.860846657490857 146.45142038074047
25.860846657490857 146.45142038074047
25.860846657490857 146.45142038074047
25.860846657490857 146.45142038074047
25.860846657490857 146.45142038074047
25.860846657490857 146.96213678199015
25.872786288306063 147.70884137787962
25.872786288306063 147.70884137787962
25.872786288306063 147.70884137787962
25.872786288306063 147.70884137787962
25.872786288306063 147.70884137787962
25.872786288306063 147.70884137787962
25.872786288306063 147.86060356895766
25.879959163992655 147.92685779241205
25.879959163992655 147.92685779241205
25.941577329653416 148.07861998349009
25.98250871347175 148.81006706060006
25.98250871347175 149.03015882689814
25.991059983646732 149.24741953975038
25.991059983646732 149.65799151981804
25.991059983646732 

49.45920638579478 281.6568204343174
49.45920638579478 282.02474169153095
49.45920638579478 282.0981056136303
49.45920638579478 282.0981056136303
49.48622096796713 282.3477696423331
49.48622096796713 282.3477696423331
49.48622096796713 282.3477696423331
49.51540647719059 282.3477696423331
49.857537243655464 282.95327440795234
49.857537243655464 283.0266383300517
49.867821481863785 283.59042678774193
49.867821481863785 283.59042678774193
49.867821481863785 283.59042678774193
49.8895475839358 284.02242321007975
49.8895475839358 284.02242321007975
49.8895475839358 284.1172115744493
49.8895475839358 284.67940471837363
49.897145908052195 285.4703553508831
49.897145908052195 285.4703553508831
49.897145908052195 285.5554984685247
49.897145908052195 285.5554984685247
49.897145908052195 285.5554984685247
49.897145908052195 285.5554984685247
49.91184298471414 285.6249296904615
49.91184298471414 286.0498557042782
50.1832734902526 286.69878727961003
50.1832734902526 286.78393039725165
50.3695202357

67.21147135681812 396.95726895072926
67.22070911621454 397.3824773843553
67.22070911621454 397.45584130645466
67.22070911621454 397.7395538609517
67.22070911621454 398.1395771092609
67.63843565510007 398.5933196838021
67.63843565510007 398.8796656735546
67.63843565510007 399.01852811742816
67.63843565510007 399.3774823275998
67.63843565510007 399.3774823275998
67.63843565510007 399.3774823275998
67.63843565510007 399.3774823275998
67.63843565510007 399.6263906546224
67.64610300353866 399.6263906546224
67.64610300353866 400.0337081342135
67.68092100562687 400.1121064031922
67.69147873065081 400.2666807427706
67.69147873065081 400.47179741009853
67.9506163512269 400.79638138454277
67.9506163512269 401.0358222413454
67.9506163512269 401.0358222413454
67.9506163512269 401.0358222413454
67.9506163512269 401.0358222413454
67.96284052850856 401.17468468521895
67.96284052850856 401.24093890867334
67.96284052850856 401.24093890867334
67.9843675487817 401.319337177652
67.9843675487817 401.319337

86.64430812560732 526.8477333963012
86.9578853798314 527.4377824814547
86.96648341013321 527.5325708458244
86.96648341013321 527.5325708458244
86.96648341013321 527.5325708458244
87.09682303299701 528.1348986129611
87.09682303299701 528.2200417306027
87.09682303299701 528.5986405366417
87.09682303299701 528.9287781860888
87.09682303299701 529.1239671868794
87.49133088110638 529.6877556445696
87.49133088110638 529.7825440089392
87.49133088110638 529.860942277918
88.18739364183355 530.5890278239086
88.18739364183355 530.6741709415502
88.23149445297472 531.2576039951078
88.23149445297472 531.8031057188865
88.23149445297472 531.8031057188865
88.23149445297472 531.8031057188865
88.23149445297472 531.8031057188865
88.23149445297472 531.8693599423409
88.23149445297472 531.8693599423409
88.23149445297472 532.093365503856
88.23149445297472 532.093365503856
88.26509710967402 532.2034113870051
88.29209858277889 532.2767753091044
88.29209858277889 532.2767753091044
88.29209858277889 532.3403960973

...

610.4488205168043 3807.162609792808
610.4488205168043 3807.162609792808
610.4488205168043 3807.162609792808
610.4488205168043 3807.162609792808
610.4488205168043 3807.299594503106
610.4488205168043 3807.4474239940214
610.4643226194978 3807.55746987717
610.5446158863008 3807.835668046788
610.5446158863008 3807.835668046788
610.5446158863008 3807.899288834987
610.8223033572737 3808.406072536074
610.8223033572737 3808.406072536074
610.8223033572737 3808.4912156537157
610.8223033572737 3808.4912156537157
610.8223033572737 3808.4912156537157
610.8383375231255 3808.6242676638512
610.8383375231255 3808.6242676638512
611.2876383220712 3809.144885527068
611.2876383220712 3809.144885527068
611.2876383220712 3809.5038397372396
611.5146667851538 3809.7239315035376
611.5146667851538 3809.7875522917366
611.5146667851538 3810.0899561222304
611.5146667851538 3810.3100478885285
611.5146667851538 3810.3100478885285
611.5146667851538 3810.3100478885285
611.5146667851538 3810.6690020987003
611.51466678515

628.496212852967 3931.2170109827434
628.5408591674346 3931.4121999835343
628.5408591674346 3931.6959125380313
628.5408591674346 3931.6959125380313
628.5523122566974 3932.2943076050055
628.6109288412679 3932.367671527105
628.6109288412679 3932.367671527105
628.6109288412679 3932.367671527105
628.6109288412679 3932.367671527105
628.6109288412679 3932.4371027490415
628.6689989730785 3932.7270801142017
628.729086835972 3932.88165445378
628.7366630073474 3932.9764428181493
628.7366630073474 3932.9764428181493
628.7366630073474 3933.2546409877673
629.0341593883201 3933.834141341162
629.0341593883201 3933.834141341162
629.0341593883201 3933.834141341162
629.0341593883201 3934.301066082717
629.0341593883201 3934.301066082717
629.0341593883201 3934.364686870916
629.16680361895 3934.4309410943706
629.1982555234841 3934.7144560306774
629.1982555234841 3934.799599148319
629.1982555234841 3934.799599148319
629.1982555234841 3935.4132517340727
629.1982555234841 3935.4132517340727
629.2058764186512 3

647.3566594337043 4058.7458956883993
647.3566594337043 4058.7458956883993
647.3566594337043 4058.8406840527687
647.3566594337043 4058.906938276223
647.3566594337043 4059.3774943931858
647.4111497006634 4059.472282757555
647.4467660913899 4059.6923745238532
647.4467660913899 4059.755995312052
647.4467660913899 4059.755995312052
647.4467660913899 4059.924147598521
647.7614971097098 4060.0092907161625
647.7614971097098 4060.6464430959522
647.7614971097098 4060.724841364931
647.7614971097098 4060.724841364931
647.7614971097098 4060.9932964006484
648.0630228097158 4061.545081840036
648.0630228097158 4061.545081840036
648.0931550830277 4061.6145130619725
648.6095567785467 4061.7979228672207
648.6095567785467 4061.9079687503695
648.6095567785467 4062.0625430899477
648.6095567785467 4062.2053382339836
648.6095567785467 4062.5642924441554
648.6095567785467 4062.5642924441554
648.6095567785467 4062.674338327304
648.6951917914375 4063.055472681426
648.7240258539384 4063.1911581268173
648.74952558

671.6252090386072 4208.000813463358
671.6252090386072 4208.000813463358
671.6505462952504 4208.290336451593
671.6715960137944 4208.368734720572
671.6715960137944 4208.602385528816
671.6988860514172 4208.680783797795
671.8422585187573 4208.8641936030435
671.8422585187573 4208.9278143912425
671.8422585187573 4209.0062126602215
671.8504823183875 4209.44795221646
671.8504823183875 4209.586814660333
671.8504823183875 4209.586814660333
671.8504823183875 4209.586814660333
671.8504823183875 4209.873160650086
671.8504823183875 4209.873160650086
671.8504823183875 4209.873160650086
671.8504823183875 4210.077994897604
672.1093527557709 4210.616426212861
672.1674961820601 4210.616426212861
672.1674961820601 4210.682680436315
672.2729991065306 4211.474407155683
672.2729991065306 4211.474407155683
672.2729991065306 4211.584453038832
672.2729991065306 4211.653884260769
672.2729991065306 4211.653884260769
672.345152942988 4211.653884260769
672.345152942988 4211.653884260769
672.345152942988 4211.653884

692.2609811823572 4337.59489345281
692.2609811823572 4337.59489345281
692.9537276280686 4338.237080179479
692.9614188885465 4338.580776870871
692.9614188885465 4339.20818611676
692.9614188885465 4339.20818611676
692.9955987668915 4339.293329234401
692.9955987668915 4339.293329234401
692.9955987668915 4339.293329234401
692.9955987668915 4339.564417705375
692.9955987668915 4339.564417705375
692.9955987668915 4339.992803137483
692.9955987668915 4340.1258551476185
692.9955987668915 4340.1258551476185
693.0302609773628 4340.1258551476185
693.0302609773628 4340.415832512779
693.0302609773628 4340.415832512779
693.491630275364 4341.05577665589
693.491630275364 4341.05577665589
693.491630275364 4341.165822539039
693.491630275364 4341.165822539039
693.491630275364 4341.239186461138
694.0588560156342 4341.9614819585695
694.0588560156342 4342.039880227549
694.0588560156342 4342.416326179828
694.0588560156342 4342.416326179828
694.0588560156342 4343.024643093947
694.0588560156342 4343.098007016046